<a href="https://www.kaggle.com/code/shaikhfaizanahmed/google-capstone-project-cyclistic?scriptVersionId=137363459" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Google Capstone Project: Cyclistic

In this case study, I performed data analysis for a fictional bike-share company in order to help them attract more riders.

First of all, I installed all the required packages and load them.

In [ ]:
# install.packages("tidyverse")
# install.packages("ggplot2")
# install.packages("skimr")
# install.packages("janitor")
# install.packages("tidyr")

library(tidyverse)
library(ggplot2)
library(skimr)
library(janitor)
library(tidyr)
library(lubridate)
library(scales)

Now I loaded the dataset using `read.csv()` function.

In [ ]:
jan2021 <- read.csv("../input/cyclistic/202101-divvy-tripdata.csv")
feb2021 <- read.csv("../input/cyclistic/202102-divvy-tripdata.csv")
mar2021 <- read.csv("../input/cyclistic/202103-divvy-tripdata.csv")
apr2021 <- read.csv("../input/cyclistic/202104-divvy-tripdata.csv")
may2021 <- read.csv("../input/cyclistic/202105-divvy-tripdata.csv")
jun2021 <- read.csv("../input/cyclistic/202106-divvy-tripdata.csv")
jly2021 <- read.csv("../input/cyclistic/202107-divvy-tripdata.csv")
aug2021 <- read.csv("../input/cyclistic/202108-divvy-tripdata.csv")
sep2021 <- read.csv("../input/cyclistic/202109-divvy-tripdata.csv")
oct2021 <- read.csv("../input/cyclistic/202110-divvy-tripdata.csv")
nov2021 <- read.csv("../input/cyclistic/202111-divvy-tripdata.csv")
dec2021 <- read.csv("../input/cyclistic/202112-divvy-tripdata.csv")

# Data Cleaning

Now the data cleaning process begins. I viewed the data frame to found its structure. `head()` function shows the first six rows of the data frame, and `str()` shows attribute's details, basically the structure of the data frame.

In [ ]:
head(jan2021)
str(jan2021)

After analyzing all the 12 data frames and there structure, I bind all in a single data frame called `cyclist_df`

In [ ]:
cyclist_df <- rbind(jan2021, feb2021, mar2021, apr2021, may2021, jun2021, jly2021, aug2021, sep2021, oct2021, nov2021, dec2021)

I also found that there are no values at some tupples, so I fill them with NA.

In [ ]:
cyclist_df[cyclist_df == ""] <- NA

Now, I calculated total rows with NA and total rows without NA.

In [ ]:
colSums(is.na(cyclist_df))
total_rows = nrow(cyclist_df)
total_rows_without_na = nrow(na.omit(cyclist_df))
na_rows = total_rows - total_rows_without_na
print(paste("Total number of rows: ",total_rows))
print(paste("Total numbers of rows without NA: ",total_rows_without_na))
print(paste("Total rows with NA: ",na_rows))


There was approximately same numbers of NA rows in each month and total number of NA rows are. So I decided to remove all the rows with NA from the data frame.

In [ ]:
cyclist_df <- na.omit(cyclist_df)
print(paste("Total rows: ",nrow(cyclist_df)))

Attributes `starting_at` and `ended_at` were of char type. I convert them to Date type.

In [ ]:
cyclist_df[, c('started_at','ended_at')] = lapply(cyclist_df[, c('started_at','ended_at')], 
                                                  function(x) as.POSIXct(x, format="%Y-%m-%d %H:%M:%S"))

Now, all our attributes are of proper type and there is no NULL or NA value in data frame.

In [ ]:
str(cyclist_df)
colSums(is.na(cyclist_df))

Now, I calculate new Atrributes of `date`,`starting_hour`, `ended_hour`, `duration_hours`, and `duration_mins`. I calculated duration by subtracting the time differnce of start and end time.

In [ ]:
cyclist_df$date <- as.Date(cyclist_df$started_at)
cyclist_df$started_hour <- lubridate::hour(cyclist_df$started_at)
cyclist_df$ended_hour <- lubridate::hour(cyclist_df$ended_at)
cyclist_df$duration_hours <- difftime(cyclist_df$ended_at, cyclist_df$started_at, units = c("hours"))
cyclist_df$duration_mins <- difftime(cyclist_df$ended_at, cyclist_df$started_at, units = c("mins"))

we get some negative time duration so I filter out the time duration which was lesser the zero.

In [ ]:
print(paste("Total rows before filtering out negative time duration: ",nrow(cyclist_df)))

cyclist_df <- cyclist_df %>% 
  filter(duration_mins > 0)

print(paste("Total rows after filtering out negative time duration: ",nrow(cyclist_df)))

Now, I cross check the NA value and attribute's type.

In [ ]:
colSums(is.na(cyclist_df))
str(cyclist_df)

By this point, data cleaning part is done. Its time to analyze the data.

# Data Analysis
## create summary data frame
I create a summary data `cyclist_df_summary` which contains count, sum of minutes, mean, minimum, maximum of the duration in minites on weekly basis.

In [ ]:
cyclist_df_summary <- cyclist_df %>% 
  group_by(Weekly = floor_date(date,"week"), started_hour) %>% 
  summarise( count = n(),
             minutes = sum(duration_mins),
             mean = mean(duration_mins),
             min = min(duration_mins),
             max = max(duration_mins)
  ) %>% 
  ungroup()

## plote of rides by date
## Summary stat : counts
### summary of hourly count

In [ ]:
summary(cyclist_df_summary$count)

### table of count by hours

In [ ]:
xtabs(cyclist_df_summary$count~cyclist_df_summary$started_hour)

In [ ]:
cyclist_df_summary$monthly <- lubridate::month(cyclist_df_summary$Weekly)


Below I plot a graph of Count of rides per day based on 28 days moving average

In [ ]:
cyclist_df_summary %>% 
  ggplot() + 
  geom_col(aes(x=Weekly, y=count)) +
  scale_y_continuous(labels = comma) +
  labs(title = "Count of rides per day",
       subtitle = "Bases on 28 days moving average",
       y = "Average rides per day")

Below is the plot of count of rides by hours, ridesper hour VS started hour

In [ ]:
cyclist_df_summary %>% 
  ggplot() +
  geom_col(aes(x=started_hour, y=count)) +
  labs(title = "Count of rides by hours",
       y = "Rides per Hour", x = "Started Hour")

## Count of rides by bike types
## Summary of bike type

In [ ]:
cyclist_df_types <- cyclist_df %>% 
  group_by(rideable_type, member_casual, weekly = floor_date(date, "week")) %>% 
  summarise( count = n(),
             minutes = sum(duration_mins),
             mean = mean(duration_mins),
             min = min(duration_mins),
             max = max(duration_mins)
  ) %>% 
  ungroup()

Below is the plot of count of rides by bike types which shows the count of each bike type i.e. clasic bike, docked bike, and electric bike.

In [ ]:
ggplot(cyclist_df_types) +
  geom_col(aes(x=weekly, y=count, fill=rideable_type)) +
  labs(title = "Count of Rides by Bike Types")

Below is the plot of count of rides by rider type which shows the count of each rider type i.e. member and casual rider.

In [ ]:
ggplot(cyclist_df_types) +
  geom_col(aes(x=weekly, y=count, fill=member_casual)) +
  labs(title = "Count of Rides by Rider Types")

Lastely, I plot the top 10 starting station with maximum number of rides.

In [ ]:
cyclist_df %>% 
  count(start_station_name, sort = TRUE) %>% 
  top_n(10) %>% 
  ggplot() +
  geom_col(aes(x=reorder(start_station_name,n), y=n)) +
  coord_flip() +
  labs(title = "Top 10 Start Station by Ride Count", x="Station Name", y="Count of Rides")
